In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
import matplotlib.pyplot as plt
import elk
import torch as t
import os
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer
from elk.extraction.prompt_dataset import PromptDataset, PromptConfig
from elk.extraction.extraction import ExtractionConfig, extract_hiddens, extract
import yaml

In [10]:
RPATH = "/fsx/home-rudolf/elk-reporters"

In [11]:
os.listdir(RPATH)

['great-mclean',
 'silly-hawking',
 'nostalgic-blackwell',
 'boring-ptolemy',
 'great-faraday',
 'brave-heisenberg',
 'strange-cartwright',
 'xenodochial-keldysh',
 'eloquent-carson',
 'interesting-poincare',
 'silly-kalam',
 'nice-lewin',
 'vigorous-germain',
 'affectionate-allen',
 'dazzling-perlman',
 'happy-einstein',
 'infallible-ellis',
 'suspicious-thompson',
 'eager-williamson',
 'pedantic-payne',
 'sweet-moser']

In [13]:
def load_config(name, rpath=RPATH):
    """Loads yaml config from RPATH/[name]/cfg.yaml"""
    with open(os.path.join(rpath, name, "cfg.yaml"), "r") as f:
        return yaml.safe_load(f)

In [55]:
load_config("sweet-moser")

{'data': {'layers': [],
  'model': 'microsoft/deberta-v2-xxlarge-mnli',
  'prompts': {'balance': False,
   'data_dir': None,
   'dataset': 'super_glue boolq',
   'label_column': None,
   'max_examples': [750, 250],
   'num_classes': None,
   'num_shots': 0,
   'num_variants': -1,
   'seed': 42},
  'token_loc': 'last'},
 'debug': False,
 'max_gpus': -1,
 'net': {'inv_weight': 5.0,
  'neg_cov_weight': 5.0,
  'num_heads': 1,
  'seed': 12,
  'var_weight': 1.0},
 'normalization': 'meanonly',
 'optim': {'lr': 0.01,
  'num_epochs': 1000,
  'num_tries': 10,
  'optimizer': 'lbfgs',
  'weight_decay': 0.01},
 'skip_baseline': False}

In [22]:
def get_relevant_reporters(model=None, dataset=None, rpath=RPATH):
    valid_reporters = []
    for name in os.listdir(rpath):
        cfg = load_config(name, rpath)
        add = True
        if model != None and cfg['data']['model'] != model:
            add = False
        if dataset != None and cfg['data']['prompts']['dataset'] != dataset:
            add = False
        if add:
            valid_reporters.append(name)
    return valid_reporters
        

In [28]:
len(get_relevant_reporters("microsoft/deberta-v2-xxlarge-mnli", "super_glue boolq"))

20

In [33]:
def num_from_str(s):
    """Strip all non-numeric characters from s and interpret as a number"""
    num_str = ''
    for char in s:
        if char.isdigit():
            num_str += char
    if num_str == '':
        return None
    else:
        return int(num_str)


In [49]:
def get_reporters_from_name(name, rpath=RPATH):
    r_dict = {
        layer_number : t.load(os.path.join(rpath, name, "reporters", layer_name))
        for (layer_number, layer_name) in [
            (num_from_str(layer_name), layer_name)
            for layer_name in os.listdir(os.path.join(rpath, name, "reporters"))
            ]
        }
    return [r_dict[i] for i in range(len(r_dict))]

In [50]:
reporters = get_reporters_from_name(get_relevant_reporters("microsoft/deberta-v2-xxlarge-mnli", "super_glue boolq")[0])

In [54]:
reporters[14]

EigenReporter()

In [47]:
model_name = "microsoft/deberta-v2-xxlarge-mnli"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/deberta-v2-xxlarge-mnli were not used when initializing DebertaV2ForMaskedLM: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v2-xxlarge-mnli and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
You should 

In [48]:
def reporter_outputs(reporters, sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to("cuda")
    outo = model(inputs["input_ids"], output_hidden_states=True)
    # the +1 in i+1 is because the first hidden state is the input embedding
    return t.stack([r(outo[1][i+1])[0] for i, r in enumerate(reporters)], dim=0)

def plot_2d_tensor_as_heatmap(t2d):
    fig, ax = plt.subplots()
    heatmap = ax.imshow(t2d.cpu().detach().numpy(), cmap="hot", interpolation="nearest")
    ax.set_xlabel("Token")
    ax.set_ylabel("Reporter (=layer)")
    plt.colorbar(heatmap)

In [52]:
reporter_outputs(reporters, "Testing")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)